## Factors of Climate Change and Climate Change Awareness

** By Sairam Sivaraj **

Climate Change is a becoming a serious issue around the world. It is affecting habitats and may lead to the extinction of animals. For this project I am going to be analysing which countries in Europe are the most environmentally friendly. I chose Europe as it is a continent that believes that climate change is a serious issue and has already taken steps in order to tackle it.

The factors that I will use to determine whether a country is environmental friendly, taking steps towards stopping Climate Change, are the Carbon Emission released/(per capita) and how much Renewable Energy the country uses.

1. I will also want examine any relationship between climate change with other factors. For instance I will see if population growth, urban growth and % of Forest Area has a relationshipwith each other. In this case % of Forest Area is used to serve as an indicator of environmentally friendliness.

2. Then I will move on Total Carbon Emissions, I will first see if it has a relationship with Total Population of a country and then if it does, I will map Carbon Emission per capita instead Total Carbon Emission for each country in Europe

3. Some countries in Europe also export oil like Norway, and therefore that might be a relationship between CO2 emission and being a exporter of oil. To see if that is the case I will compare the countries in Europe with countries that export oil.

4. Finally I will map out the Renewable Energy Consumption for each country in Europe and compare it with the Renewable Energy Consumption of oil exporting countries. I will also see if Renewable Energy Consumption has a relation with Tax, as I believe countries with higher taxes, value their society more than countries with low taxes, who value individuals more.

In [1]:
# Packages needed including Advanced Plotly functions
import pandas as pd                    # data package 
import datetime as dt                  # date and time module
import numpy as np                     # foundation for Pandas

#WB functions
import wbdata
from pandas_datareader import wb       # worldbank data

# Web Scraping
import requests
from bs4 import BeautifulSoup

# plotly imports
from plotly.offline import iplot, iplot_mpl  # plotting functions
import plotly.graph_objs as go               # ditto
import plotly                                 # just to print version and init notebook
import plotly.plotly as py
import math
# these lines make our graphics show up in the notebook
%matplotlib inline             
plotly.offline.init_notebook_mode(connected=True)

I will be getting my data through the World Bank data and mapping it. Thus for mapping I will need the ISO codes and Longtiude and Latitude data will be needed. Hence I will just get the data for the Entire World, even though I will be focusing on Europe.



In [2]:
# read a list of latitude and longitude coordinates for
# country capitals
lat_lon = pd.read_html("http://www.csgnetwork.com/llinfotable.html", header=0, 
                       attrs={"align": "center", "cellpadding": 5, "bgcolor": "#FFFFFF"})[0]

In [3]:
# Changing the name of United Kingdom
lat_lon['Country'] = lat_lon['Country'].replace(
        to_replace=['United Kingdom of Great Britain and Northern Ireland'],
        value=['United Kingdom'])

In [4]:
# clean up so lat and long are numeric in degrees east and degrees north
def clean_latlon(series, to_negate):
    # get XX.YY data
    split1 = series.str.split("°")
    split2 = split1.str.get(1).str.split("'")
    data = split1.str.get(0) + (split2.str.get(0).astype(float)/60).astype(str).str.lstrip('0')
    
    # now add a negative side if last character == to_negate
    signs = split2.str.get(1) == to_negate
    signs = signs.replace({True: "-", False: ""})
    
    data = signs.str[:] + data.str[:]
    return data.astype(float)
    
lat_lon["Latitude"] = clean_latlon(lat_lon["Latitude"], "S")
lat_lon["Longitude"] = clean_latlon(lat_lon["Longitude"], "W")

#Dropping the capital

lat_lon = lat_lon.drop("Capital", axis=1)
lat_lon = lat_lon.set_index("Country")

In [5]:
#Checking the data
lat_lon.head()

,Latitude,Longitude
Country,,
Afghanistan,34.466667,69.183333
Albania,41.300000,19.816667
Algeria,36.700000,3.133333
American Samoa,-14.266667,-170.716667
Andorra,42.516667,1.533333


In [6]:
# Now, we need to create a Dataframe of country names and iso codes, hence we will get the iso codes from the below URL
url = 'https://unstats.un.org/unsd/methodology/m49/'
iso_raw = requests.get(url)
iso_soup = BeautifulSoup(iso_raw.content, 'html.parser')

In [7]:
iso = pd.read_html(str(iso_soup.find_all('table')[0]), header=0)[0]

In [8]:
iso = iso.rename(columns={"ISO-alpha3 code": "ISO", 
                          "Country or Area": "Country"})
iso = iso.drop("M49 code", axis=1)
iso['Country'] = iso['Country'].replace(
                    to_replace=['United Kingdom of Great Britain and Northern Ireland'],
                    value=['United Kingdom'])
iso = iso.set_index("Country")
iso.head()



,ISO
Country,
Afghanistan,AFG
Åland Islands,ALA
Albania,ALB
Algeria,DZA
American Samoa,ASM


In [9]:
# Merging the two tables (ISO codes with Latitude and Longitude. This will be required later for the map graphs
iso = iso.merge(lat_lon, left_index=True, right_index=True)
iso.head()

,ISO,Latitude,Longitude
Country,,,
Afghanistan,AFG,34.466667,69.183333
Albania,ALB,41.300000,19.816667
Algeria,DZA,36.700000,3.133333
American Samoa,ASM,-14.266667,-170.716667
Andorra,AND,42.516667,1.533333


I have now created a table with the ISO codes and Latitude/Longitude for each country in the world, using their capital. I will now move onto seeing if their is a relationship between Population Growth, Urban Population Growth and how much of the land in these countries is Forest Area.

I am doing this as Population Growth and Urban Population seem to be factors that affect CO2 emissions, and how much Forest Area a land has. To accurately map the relaitonship I will be using sample countries from Europe.

In [10]:
#Searching indicator names in WB

wbdata.search_indicators("Population Growth")

#using the above search funtion, we can find out 
#SP.URB.GROW is for annual urban population growth and SP.POP.GROW is for annual population growth
#AG.LND.FRST.ZS is the Forest area (% of land area)

SP.URB.GROW   	Urban population growth (annual %)
SP.RUR.TOTL.ZG	Rural population growth (annual %)
SP.POP.GROW   	Population growth (annual %)


In [11]:
# The year I will be using is 2013 since all the indicators I will be using for this project have data for this year.
year = 2013

# The sample countries in Europe that I will be using
sample_europe = ["Sweden", "United Kingdom", "Netherlands", "Luxembourg", "Germany", "Austria"]
iso_europe = iso[iso.index.isin(sample_europe)]

# Dataframe for Urban Growth
dfurb = wb.download(country=iso_europe["ISO"], indicator='SP.URB.GROW', 
                         start=year, end=year)
dfurb = dfurb.reset_index(level="year") 
dfurb = dfurb.drop('year',1)
dfurb = dfurb.rename(columns = {'SP.URB.GROW': 'Urban Growth (%)'})

# Dataframe for Population Growth
dfpop = wb.download(country=iso_europe["ISO"], indicator='SP.POP.GROW', 
                         start=year, end=year)
dfpop = dfpop.reset_index(level="year") 
dfpop = dfpop.drop('year',1)
dfpop = dfpop.rename(columns = {'SP.POP.GROW': 'Population Growth (%)'})

# Dataframe for forest area
dffor = wb.download(country=iso_europe["ISO"], indicator='AG.LND.FRST.ZS', 
                         start=year, end=year)
dffor = dffor.reset_index(level="year") 
dffor = dffor.drop('year',1)
dffor = dffor.rename(columns = {'AG.LND.FRST.ZS': 'Forest Area (% of Total Area)'})

In [12]:
#Merging the 3 dataframes together
euro = [dfurb, dffor, dfpop]
dfeuro = pd.concat(euro, axis=1,join='inner')


In [13]:
# Viewing the dataframe
dfeuro

,Urban Growth (%),Forest Area (% of Total Area),Population Growth (%)
country,,,
Austria,0.614462,46.835734,0.584104
Germany,0.542994,32.718986,0.272900
United Kingdom,0.984309,12.854958,0.669534
Luxembourg,2.670351,33.474903,2.311176
Netherlands,1.077518,11.124963,0.294821
Sweden,1.024086,68.917857,0.847349


In [14]:
# Next I will create a bar graph to better analyse the data and relationship
urb = go.Bar(
    x=dfeuro.index,
    y=dfeuro['Urban Growth (%)'],
    name='Urban Growth (%)',
    marker=dict(
        color='rgb(49,130,189)'
    )
)
pop = go.Bar(
    x=dfeuro.index,
    y=dfeuro['Population Growth (%)'],
    name='Population Growth (%)',
    marker=dict(
        color='rgb(204,204,204)',
    )
)

forest = go.Bar(
    x=dfeuro.index,
    y=dfeuro['Forest Area (% of Total Area)'],
    name='Forest Area (% of Total Area))',
    marker=dict(
        color='rgb(255,140,0)',
    )
)

layout = {
  'xaxis': {'title': 'Country'},
  'yaxis': {'title': 'Number'},
  'title': 'Forest Area, Urban and Population Growth'
};


iplot(go.Figure(data=[pop, urb, forest], layout = layout))

It is hard to see the relationship between the three indicators as Forest Area (% of Total Area) is much higher than the other two. 

In [15]:
# As the graph is unclear due to the percent of forest area being so high in comparison to the other values. 
# Thus as we are looking only for a relationship and not exact numbers I am going to divide the forest area values by 20

dfeuro['Forest Area (% of Total Area)/20'] = dfeuro['Forest Area (% of Total Area)']/20 

In [16]:
# Now I will graph the data again but use the Forest Area/20 instead of Forest Area
urb = go.Bar(
    x=dfeuro.index,
    y=dfeuro['Urban Growth (%)'],
    name='Urban Growth (%)',
    marker=dict(
        color='rgb(49,130,189)'
    )
)
pop = go.Bar(
    x=dfeuro.index,
    y=dfeuro['Population Growth (%)'],
    name='Population Growth (%)',
    marker=dict(
        color='rgb(204,204,204)',
    )
)

forest = go.Bar(
    x=dfeuro.index,
    y=dfeuro['Forest Area (% of Total Area)/20'],
    name='Forest Area (% of Total Area)/20',
    marker=dict(
        color='rgb(255,140,0)',
    )
)


layout = {
  'xaxis': {'title': 'Country'},
  'yaxis': {'title': 'Number'},
  'title': 'Forest Area, Urban and Population Growth'
};


iplot(go.Figure(data=[pop, urb, forest], layout = layout))

It is unclear if Population Growth and Urban Growth are related. However in Europe it seems tha Urban Growth is always higher than Population Growth, at least for the samaple countries I have taken. Also suprisingly, when the difference between Urban Growth and Population growth seems to be large like in the Netherlands, the Forest Area (%) seems to be low. However when the difference is small like in Austria and Sweden the Forest Area (%) seems to be high. Therefore, there seems to be a relation. However, as I do not account for other factors which include geograhical location, I can not be 100% sure

I will now move onto seeing the CO2 emission in Europe for 2013, but first I will make a dataframe containing all the information I will be using for countries in Europe.

In [17]:
# Now it is time to see if there CO2 emissions and renewable Energy usage in countries in Europe.
# Remeber we have found the iso codes so now it is time to work on creating another dataset

# Hence we will be using the WB search indicator function again
wbdata.search_indicators("tax revenue")

#Using that we know the indicator code for total renewable energy consumption is 3.1_RE.CONSUMPTION
# carbon emissions per kt is EN.ATM.CO2E.KT, population, total is SP.POP.TOTL and alternative energy is EG.USE.COMM.CL.ZS 

GC.TAX.TOTL.GD.ZS	Tax revenue (% of GDP)
GC.TAX.TOTL.CN   	Tax revenue (current LCU)
GC.TAX.IMPT.ZS   	Customs and other import duties (% of tax revenue)
GC.TAX.EXPT.ZS   	Taxes on exports (% of tax revenue)
REV.TXRV.SHR.CR  	Total Tax Revenue Sharing/DBH Pajak (in IDR)


In [18]:
# We will list all the countries in Europe and get the approriate data for each country

def euro_wb_data(indicators, year=2013):# get data from worldbank
    europe = ["Albania",  "Andorra",  "Armenia",  "Austria",  "Azerbaijan",  "Belarus",
              "Belgium",  "Bosnia and Herzegovina",  "Bulgaria",  "Croatia",  "Cyprus",
              "Czech Republic",  "Denmark",  "Estonia",  "Finland",  "France",  "Georgia",
              "Germany",  "Greece",  "Hungary",  "Iceland",  "Ireland",  "Italy",
              "Kazakhstan",  "Kosovo",  "Latvia",  "Liechtenstein",  "Lithuania",
              "Luxembourg",  "Macedonia",  "Malta",  "Moldova",  "Monaco",  "Montenegro",
              "Netherlands",  "Norway",  "Poland",  "Portugal",  "Romania",  "Russia",  
              "San Marino",  "Serbia",  "Slovakia",  "Slovenia",  "Spain",  "Sweden",
              "Switzerland",  "Turkey",  "Ukraine",  "United Kingdom",
              "Vatican City"]

    iso_europe = iso[iso.index.isin(europe)]

    # IP.JRN.ARTC.SC is "scientific and technical journal articles"
    # NOTE: visit the world bank website to pick a different subject if you'd like
    # link: http://data.worldbank.org/indicator
    df = wb.download(country=iso_europe["ISO"], indicator=indicators, 
                         start=year, end=year)
    df = df.reset_index(level="year")
    df.index.name = "Country"

    # some countries didn't have data. Drop them now
    df = df.dropna()
    
    return df


world = euro_wb_data(["SP.POP.TOTL" , "EN.ATM.CO2E.KT", "EG.FEC.RNEW.ZS","GC.TAX.TOTL.GD.ZS"])
world.rename(columns={"SP.POP.TOTL": "Total Population", "EN.ATM.CO2E.KT": 'Total Carbon Emissions (KT)',
                     "EG.FEC.RNEW.ZS": "Renewable energy consumption (% Total Energy consumption)",
                      "GC.TAX.TOTL.GD.ZS": "Tax Revenue (% of GDP)"}, inplace=True)
world.head()

,year,Total Population,Total Carbon Emissions (KT),Renewable energy consumption (% Total Energy consumption),Tax Revenue (% of GDP)
Country,,,,,
Albania,2013,2896652,4814.771,41.288978,16.497139
Austria,2013,8479375,62408.673,34.687002,26.527452
Azerbaijan,2013,9416801,35643.240,2.526095,13.498255
Belgium,2013,11182817,93618.510,8.119471,26.211885
Bosnia and Herzegovina,2013,3823533,21906.658,19.477394,19.777889


In [19]:
world = world.drop('year',1) # As we are taking data only from 2013, the year column is unnecessary and therefore we will drop it


In [20]:
# Merge the new database with the iso database in order to have latitude and longitude
world = world.merge(iso, left_index=True, right_index=True)
world.head()

,Total Population,Total Carbon Emissions (KT),Renewable energy consumption (% Total Energy consumption),Tax Revenue (% of GDP),ISO,Latitude,Longitude
Country,,,,,,,
Albania,2896652,4814.771,41.288978,16.497139,ALB,41.300000,19.816667
Austria,8479375,62408.673,34.687002,26.527452,AUT,48.200000,16.366667
Azerbaijan,9416801,35643.240,2.526095,13.498255,AZE,40.483333,49.933333
Belarus,9465997,63769.130,7.009208,13.798910,BLR,53.866667,27.500000
Belgium,11182817,93618.510,8.119471,26.211885,BEL,50.850000,4.350000


I have now created a dataframe with all countries in Europe and the indicators I will be using. I know must determine whether to map Total Carbon Emissions or Carbon Emission per Capita. For that I will first see if there is a relationship between Total Population of a country and its Total Carbon Emissions 

In [21]:
# First I will try to see if there is a correlation between Total Population and Total Carbon Emissions
# To do this I will make a scatter plot with both these variables.

traceeuro = go.Scatter(
    x=world['Total Population'],
    y=world['Total Carbon Emissions (KT)'],
    mode='markers',
    name='Europe',
    text=world.index,
    marker=dict(
        sizemode='diameter',
        sizeref=0.85,
        size=20,
        line=dict(
            width=2
        ),
    )
)


data1 = [traceeuro]
layout1 = go.Layout(
    title='Total Carbon Emissions vs Total Population',
    xaxis=dict(
        title='Total Population',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Total Carbon Emissions(kt)',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig1 = go.Figure(data=data1, layout=layout1)
iplot(fig1)

** With a few exceptions like Turkey, it is clear to see that when Total Population increases, Total Carbon Emissions increases, and this makes sense since there are more people.**

A majority of countries in Europe have a low population and total carbon emission. In order to analyse the large cluster that has a total population lower than 20M, I will be regraphing the data in that region.

In [22]:
traceeuro = go.Scatter(
    x=world['Total Population'],
    y=world['Total Carbon Emissions (KT)'],
    mode='markers',
    name='Europe',
    text=world.index,
    marker=dict(
        sizemode='diameter',
        sizeref=0.85,
        size=20,
        line=dict(
            width=2
        ),
    )
)


data1 = [traceeuro]
layout1 = go.Layout(
    title='Total Carbon Emissions vs Total Population for first 20M',
    xaxis=dict(
        title='Total Population',
        range=[0,22000000],
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Total Carbon Emissions(kt)',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig1 = go.Figure(data=data1, layout=layout1)
iplot(fig1)

When the population is under 20M, the total Carbon Emission seems to be nearly the same for all countries with the exemption of Netherlands and Kazakhstan. This could be due to other factors, for instance Kazakhstan is a exporter of oil and this is a factor we will be considering later on.

As Population affects Total Carbon Emissions. I am now going to find the Carbon Emission per capita and then map the countries in Europe using CO2 per capita as it accounts for the difference in population between countries.

In [23]:
# Create Carbon Emission per capita table
world['Carbon Emission per capita'] = world['Total Carbon Emissions (KT)']/world['Total Population']
world.head()

,Total Population,Total Carbon Emissions (KT),Renewable energy consumption (% Total Energy consumption),Tax Revenue (% of GDP),ISO,Latitude,Longitude,Carbon Emission per capita
Country,,,,,,,,
Albania,2896652,4814.771,41.288978,16.497139,ALB,41.300000,19.816667,0.001662
Austria,8479375,62408.673,34.687002,26.527452,AUT,48.200000,16.366667,0.007360
Azerbaijan,9416801,35643.240,2.526095,13.498255,AZE,40.483333,49.933333,0.003785
Belarus,9465997,63769.130,7.009208,13.798910,BLR,53.866667,27.500000,0.006737
Belgium,11182817,93618.510,8.119471,26.211885,BEL,50.850000,4.350000,0.008372


In [24]:
# Now map out the Carbon Emission per capita
data2 = [ dict(
        type = 'choropleth',
        locations = world['ISO'],
        z = world['Carbon Emission per capita'],
        text = world.index,
        colorscale = [[0,"rgb(172, 10, 5)"],[0.35,"rgb(190, 60, 40)"],[0.5,"rgb(245, 100, 70)"],\
            [0.6,"rgb(245, 120, 90)"],[0.7,"rgb(247, 137, 106)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            title = 'Carbon Emission per capita'),
      ) ]

layout2 = dict(
    title = 'Carbon Emission per Capita(2013) in Europe',
    geo = dict(
        scope = 'europe',
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig2 = dict( data=data2, layout=layout2)
iplot(fig2, validate=False)

For this graph, we can see in the countries that emit the most Carbon Emissions per capita in Europe are Norway and Estonia, while Finland and Germany follow after them. Places like Sweden, Portugal and Spain emit comparatively less Carbon Emissions per capita. 

The data we mapped is on a comparative basis and only compares countries in Europe. Therefore I believe we should also analyse the carbon emissions for some countries not in Europe. I will choose a sample of countries who all export oil. The reason I am doing this is for two reasons, 1. to give a comparison with countries not in Europe, and 2. it may explain why Norway has a high carbon emission per capita, as Norway exports a lot of oil.

In [25]:
# Let us see if there is an indicator to use, to see which country exports a lot of oil
wbdata.search_indicators("Oil")

CRUDE_WTI                  	Crude oil, WTI, $/bbl, current$
CRUDE_PETRO                	Crude oil, avg, spot, $/bbl, current$
CRUDE_DUBAI                	Crude oil, Dubai, $/bbl, current$
CRUDE_BRENT                	Crude oil, Brendt, $/bbl, current$
COCONUT_OIL                	Coconut oil, $/mt, current$
EN.ATM.CO2E.EG.ZS          	CO2 intensity (kg per kg of oil equivalent energy use)
EG.USE.PCAP.KG.OE          	Energy use (kg of oil equivalent per capita)
EG.USE.COMM.GD.PP.KD       	Energy use (kg of oil equivalent) per $1,000 GDP (constant 2011 PPP)
EG.IMP.TOTL.KT.OE          	Energy imports (kt of oil equivalent)
EG.GDP.PUSE.KO.PP.KD       	GDP per unit of energy use (constant 2011 PPP $ per kg of oil equivalent)
EG.GDP.PUSE.KO.PP          	GDP per unit of energy use (PPP $ per kg of oil equivalent)
EG.ELC.PETR.ZS             	Electricity production from oil sources (% of total)
EG.ELC.FOSL.ZS             	Electricity production from oil, gas and coal sources (% of total)
IC.FRM.I

In [26]:
year = 2013
dfoil = wb.download(country='all', indicator='EG.ELC.PETR.ZS', 
                         start=year, end=year)
dfoil = dfoil.dropna()


In [27]:
dfoil.sort_values('EG.ELC.PETR.ZS',ascending= False).head()


,,EG.ELC.PETR.ZS
country,year,
Gibraltar,2013,100.000000
South Sudan,2013,99.576271
Eritrea,2013,99.449036
Benin,2013,99.421965
Malta,2013,98.445135


The following data doesn't seem like an accurate indicator as the largest exporter of oil is Saudia Arabia and this list does not include it.

Therefore instead I will be using the following [data](http://www.worldstopexports.com/worlds-top-oil-exports-country/). The year is 2016, which is different however I believe it won't matter since I am justing trying to see which countries export a lot of oil and not the exact amount exported.

In [28]:
# Finding the appropriate data for oil exporting countries using the World Bank

def oil_data(indicators, year=2013):# get data from worldbank
    oilexp = ["Kuwait", "Saudi Arabia", "Qatar", "Oman", "Bahrain", "United Arab Emirates", "Russian Federation", 
              "Canada", "Iraq", "Norway",'Iran']

    iso_oil = iso[iso.index.isin(oilexp)]

    dfoil = wb.download(country=iso_oil["ISO"], indicator=indicators, 
                         start=year, end=year)
    dfoil = dfoil.reset_index(level="year")
    dfoil.index.name = "Country"

    
    return dfoil


oil = oil_data(["SP.POP.TOTL" , "EN.ATM.CO2E.KT", "EG.FEC.RNEW.ZS","GC.TAX.TOTL.GD.ZS"])
oil.rename(columns={"SP.POP.TOTL": "Total Population", "EN.ATM.CO2E.KT": 'Total Carbon Emissions (KT)',
                     "EG.FEC.RNEW.ZS": "Renewable energy consumption (% Total Energy consumption)",
                      "GC.TAX.TOTL.GD.ZS": "Tax Revenue (% of GDP)"}, inplace=True)
oil

,year,Total Population,Total Carbon Emissions (KT),Renewable energy consumption (% Total Energy consumption),Tax Revenue (% of GDP)
Country,,,,,
United Arab Emirates,2013,9039978,169122.040,0.132213,0.371889
Bahrain,2013,1349427,31957.905,0.000000,1.069896
Canada,2013,35155451,475734.578,23.045100,11.652399
Iraq,2013,34107366,167812.921,1.368784,NaN
Kuwait,2013,3593689,97960.238,0.000000,0.777451
Norway,2013,5079623,59636.421,57.693465,25.113427
Oman,2013,3906912,61183.895,0.000000,2.523722
Qatar,2013,2101288,85023.062,0.000000,NaN
Russian Federation,2013,143506911,1789074.295,3.666604,13.292079


In [29]:
# Dropping the year column as it is unnecessary and finding the carbon emission per capita
oil = oil.drop('year',1)
oil['Carbon Emission per Capita'] = oil['Total Carbon Emissions (KT)']/oil['Total Population']

In [30]:
# I am also sorting out the data, starting from the highest Carbon Emission per capita to the lowest
oil = oil.sort_values('Carbon Emission per Capita',ascending= False)

In [31]:
oil['Carbon Emission per Capita']

Country
Qatar                   0.040462
Kuwait                  0.027259
Bahrain                 0.023683
United Arab Emirates    0.018708
Saudi Arabia            0.017927
Oman                    0.015660
Canada                  0.013532
Russian Federation      0.012467
Norway                  0.011740
Iraq                    0.004920
Name: Carbon Emission per Capita, dtype: float64

In [32]:
# Sorting out the data. Showing the 5 countries in Europe that have the highest carbon emission per capita.
worldcar = world.sort_values('Carbon Emission per capita',ascending= False).head()
worldcar['Carbon Emission per capita'].head()

Country
Luxembourg     0.018701
Kazakhstan     0.015433
Estonia        0.015110
Norway         0.011740
Netherlands    0.010115
Name: Carbon Emission per capita, dtype: float64

Here you can clearly see that the carbon emission per capita for countries that export oil in generally is higher than the countries that do not with the exemptiom of Iraq. This might also explains why Norway, an exporter of oil, has a high CO2 emission per capita unlike its neighbour Sweden.

Now, that we have mapped the Carbon Emission per capita for countries in Europe and established that it is affected by whether the country exports oil, we can now move onto renewable energy.

In [33]:
# Mapping Renewable Energy data. Doesn't have to be per capita as it is already in % of Total Energy Consumption
data3 = [ dict(
        type = 'choropleth',
        locations = world['ISO'],
        z = world['Renewable energy consumption (% Total Energy consumption)'],
        text = world.index,
        colorscale = [[0,"rgb(5, 10, 172)"],[0.35,"rgb(40, 60, 190)"],[0.5,"rgb(70, 100, 245)"],\
            [0.6,"rgb(90, 120, 245)"],[0.7,"rgb(106, 137, 247)"],[1,"rgb(220, 220, 220)"]],
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            ticksuffix = '(%)',
            title = 'RE Consumption (% TE consumption)'),
      ) ]

layout3 = dict(
    title = 'Renewable Energy Consumption (2013) in Europe',
    geo = dict(
        scope = 'europe',
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig3 = dict( data=data3, layout=layout3)
iplot(fig3, validate=False)

**Suprisingly Norway, Sweden and Iceland have the highest RE consumption. This is interesting as Norway also had a high CO2 emission rate, therefore either Norway uses a lot of energy, or its CO2 emission stem from the fact it exports oil/other sources**

I will try to see if renewable energy consumption is affected by factors like tax rate. The reason I am using tax rate is because I believe countries that have a higher tax rate, have more collectivism ideas rather than individualism. Thus, they will be more environmental friendly. Again I will first primarily focus on Europe.

I will make a bubble chart again and set the size of the bubbles according to the population, to depict population of the country as well

In [34]:
traceeuro = go.Scatter(
    x=world['Tax Revenue (% of GDP)'],
    y=world['Renewable energy consumption (% Total Energy consumption)'],
    mode='markers',
    name='Europe',
    text=world.index,
    marker=dict(
        sizemode='diameter',
        color = np.random.randn(500), #set color equal to a variable
        colorscale='Viridis',
        sizeref=0.85,
        size=world['Total Population']/800000,
        line=dict(
            width=2
        ),
    )
)


data4 = [traceeuro]
layout4 = go.Layout(
    title='Renewable Energy Consumption vs Tax Rate',
    xaxis=dict(
        title='Tax Rate (% of GDP)',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Renewable energy consumption (% Total Energy consumption)',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig4 = go.Figure(data=data4, layout=layout4)
iplot(fig4)

If the bubbles are set according to the population see we can't clearly see the relationship clearly as most of the consumption and tax rate is in one cluster. Thus, I will move onto setting it into a normalised size.

In [35]:
traceeuro = go.Scatter(
    x=world['Tax Revenue (% of GDP)'],
    y=world['Renewable energy consumption (% Total Energy consumption)'],
    mode='markers',
    name='Europe',
    text=world.index,
    marker=dict(
        sizemode='diameter',
        color = np.random.randn(500), #set color equal to a variable
        colorscale='Viridis',
        sizeref=0.85,
        size=25,
        line=dict(
            width=2
        ),
    )
)


data4 = [traceeuro]
layout4 = go.Layout(
    title='Renewable Energy Consumption vs Tax Rate',
    xaxis=dict(
        title='Tax Rate (% of GDP)',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Renewable energy consumption (% Total Energy consumption)',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig4 = go.Figure(data=data4, layout=layout4)
iplot(fig4)

Here, the graph seems to be clearly, however it looks like my hypothesis is wrong because there doesnt seem to be a relationship between Renewable Energy consumption and tax. Instead it could be due to Geographical location, as countries as Sweden, Norway and Finlan which are all close together all have a high Renewable Energy Consumption. Also since I only used countries in Europe, where most countries generally value the environment, regardless of Tax Rate, a relationship can not been seen. To discuss this point I will depict the Renewable Energy Consumption of countries that export oil, not in Europe.

In [36]:
Reoil=oil[['Tax Revenue (% of GDP)', 'Renewable energy consumption (% Total Energy consumption)']]
Reoil

,Tax Revenue (% of GDP),Renewable energy consumption (% Total Energy consumption)
Country,,
Qatar,NaN,0.000000
Kuwait,0.777451,0.000000
Bahrain,1.069896,0.000000
United Arab Emirates,0.371889,0.132213
Saudi Arabia,NaN,0.006374
Oman,2.523722,0.000000
Canada,11.652399,23.045100
Russian Federation,13.292079,3.666604
Norway,25.113427,57.693465


Here with the exemption of Norway, as it is in Europe, the tax revenue of these countries is low and so is the renewable energy consumption. However, while tax may seem like a good indicator, it doesn't seem accurate. As Russia has a high tax but low RE consumption and Canda has a relatively low tax while relatively high RE consumption. Therefore it is rather the values the area/part of the world exhibits and these values are not represented by Tax Rate but probably instead by Geographical location.

## Conclusion

Climate change is an enormously complex issue. It is affected by economics, cultural habits, certain values and many other stuff. In this report I chose not to focus on the temeperature levels and increase in temperature of countries in Europe but rather find some of these factors. While this notebook obviously does not begin to scratch the surface of climate change data and takes on many assumptions, I still believe it was able to find certain relationships.


1. Countries in Europe that have a high difference between Urban growth and Population growth have a comparatively less Forest Area

2. Countries that export Oil generally emit more CO2 emissions than countries that do not.

3. Higher populated countries have higher total carbon emissions

4. Countries in Europe that have a high tax rate do not necessarily have a high Renewable Energy Consumption.

We also found that Sweden seems to be the most climate change aware and environmentally friendly country in Europe as it has low CO2 emissions per capita and high Renewable Energy consumption (% of Total Energy)